# Курсовой проект по теме "Библиотеки Python для Data Science: Numpy, Matplotlib, Scikit-learn"

### 1. Загрузим необходимые библиотеки, файлы, скрипты и модули, которые понадобятся нам в проекте.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse, r2_score as r2
from sklearn.model_selection import KFold

In [2]:
train_directory = '/home/maxim/Документы/GeekBrains/1 четверть. Библиотки Python для Data Science. Numpy, Matplotlib, Scikit-learn/Manuals/train.csv'

In [3]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

### 2. EDA

In [4]:
df = pd.read_csv(train_directory)

In [5]:
data_to_change_type = df.select_dtypes(include='object').keys()
for column in data_to_change_type:
    old_values = np.sort((pd.unique(df[column])))
    new_values = [num for num in range(0, len(old_values))]
    for val in range(len(pd.unique(df[column]))):
        df[column] = np.where(df[column] == old_values[val], new_values[val], df[column])
    df[column] = df[column].astype(float)

In [ ]:
df.hist(figsize=(20,20), bins=20)

array([[<AxesSubplot:title={'center':'Id'}>,
        <AxesSubplot:title={'center':'DistrictId'}>,
        <AxesSubplot:title={'center':'Rooms'}>,
        <AxesSubplot:title={'center':'Square'}>],
       [<AxesSubplot:title={'center':'LifeSquare'}>,
        <AxesSubplot:title={'center':'KitchenSquare'}>,
        <AxesSubplot:title={'center':'Floor'}>,
        <AxesSubplot:title={'center':'HouseFloor'}>],
       [<AxesSubplot:title={'center':'HouseYear'}>,
        <AxesSubplot:title={'center':'Ecology_1'}>,
        <AxesSubplot:title={'center':'Ecology_2'}>,
        <AxesSubplot:title={'center':'Ecology_3'}>],
       [<AxesSubplot:title={'center':'Social_1'}>,
        <AxesSubplot:title={'center':'Social_2'}>,
        <AxesSubplot:title={'center':'Social_3'}>,
        <AxesSubplot:title={'center':'Healthcare_1'}>],
       [<AxesSubplot:title={'center':'Helthcare_2'}>,
        <AxesSubplot:title={'center':'Shops_1'}>,
        <AxesSubplot:title={'center':'Shops_2'}>,
        <AxesSubplot:

In [ ]:
df.describe()

In [ ]:
df.info()

### **Параметр Rooms**

In [ ]:
df['Rooms'].value_counts()

#### Заменим значения, которые больше 6 на моду, а те, которые равны 0 на 1.

### **Параметр Square**

In [ ]:
plt.scatter(df['Square'], df['Id'])

In [ ]:
(df['Square'] > 300).sum()

In [ ]:
(df['Square'] < 10).sum()

#### Заменим значения, которые больше 300 на среднее значение, а те, которые меньше 10 на медианные.

### **Параметр LifeSquare**

In [ ]:
(df['LifeSquare'] > 250).sum()

#### Эти значения заменим на средние.

In [ ]:
(df['LifeSquare'] < 10).sum()

#### Эти значения заменим на медианные.

In [ ]:
(df['LifeSquare'].isnull()).sum()

In [ ]:
((df['LifeSquare'] + df['KitchenSquare']) > df['Square']).sum()

#### Эти значения заменим на среднее соотношение жилплощади к общей.

### **Параметр KitchenSquare**

In [ ]:
df['KitchenSquare'].value_counts()

In [ ]:
(df['KitchenSquare'] < 2).sum()

#### Изменим значения на 2.

In [ ]:
(df['KitchenSquare'] > 50).sum()

#### Эти значения меняем на медиану.

### **Параметр HouseFloor**

In [ ]:
(df['Floor'] > df['HouseFloor']).sum()

#### Заменим значения HouseFloor, где Floor < 5 на 5, где < 10 на 10, где < 15 на 15, где < 20 на 20, а остальные поменяем на значение Floor.

In [ ]:
(df['HouseFloor'] > 60).sum()

#### Заменим на значение 60.

In [ ]:
df['HouseFloor'].value_counts()

### **Параметр HouseYear**

In [ ]:
(df['HouseYear'] > 2020).sum()

#### Меняем на медианные значения.

### **Healthcare_1**

In [ ]:
(df['Healthcare_1'].isna()).sum()

#### Параметр удаляем.

In [ ]:
class DataPipeline:
    
    """Подготовка исходных данных. Заполнение пропусков, правка некорректных значений."""
    
    def __init__(self):
        
        """Параметры класса"""
        self.rooms_mode = None
        self.square_mean = None
        self.square_median = None
        self.lifesquare_median = None
        self.lifesquare_mean = None
        self.life_to_square = None
        self.kitchensquare_median = None
        self.houseyear_median = None
        self.medians = None
        
    def fit(self, df):
        
        """Сохранение статистик"""
        self.rooms_mode = df['Rooms'].mode()[0]
        self.square_mean = df['Square'].mean()
        self.square_median = df['Square'].median()
        self.lifesquare_mean = df['LifeSquare'].mean()
        self.lifesquare_median = df['LifeSquare'].median()
        self.life_to_square = df['LifeSquare'].median() / df['Square'].median()
        self.kitchensquare_median = df['KitchenSquare'].median()
        self.houseyear_median = df['HouseYear'].median()
        self.medians = df.median()
    
    def transform(self, df):
        
        """Трансформация данных"""
        
        # 1. Заменим буквенные значения в выборке на цифренные.
        data_to_change_type = df.select_dtypes(include='object').keys()
        for column in data_to_change_type:
            old_values = np.sort((pd.unique(df[column])))
            new_values = [num for num in range(0, len(old_values))]
            for val in range(len(pd.unique(df[column]))):
                df[column] = np.where(df[column] == old_values[val], new_values[val], df[column])
            df[column] = df[column].astype(int)

        # 2. Введем новую фичу, которая будет показывать, сколько раз был изменен объект недвижимости в ходе обработки данных.
        df['TimesChanged'] = 0
        
        # 3. Параметр Rooms. Заменим значения, которые больше 6 на моду, а те, которые равны 0 на 1.
        df['TimesChanged'] = np.where(((df['Rooms'] == 0) | (df['Rooms'] > 6)), (df['TimesChanged'] + 1), df['TimesChanged'])
        df['Rooms'] = np.where((df['Rooms'] == 0), 1, df['Rooms'])
        df['Rooms'] = np.where((df['Rooms'] > 6), self.rooms_mode, df['Rooms'])
        
        # 4. Параметр Square. Заменим значения, которые больше 300 на среднее значение, а те, которые меньше 10 на медианные.
        df['TimesChanged'] = np.where(((df['Square'] < 10) | (df['Square'] > 300)), (df['TimesChanged'] + 1), df['TimesChanged'])
        df['Square'] = np.where((df['Square'] < 10), self.square_median, df['Square'])
        df['Square'] = np.where((df['Square'] > 300), self.square_mean, df['Square'])
        
        # 5. Параметр LifeSquare. Заменим значения, которые больше 250 на среднее значение, а те, которые меньше 10 на медианные.
        # Значения NaN и значения, где жилплощадь + кухня больше общей заменим на среднее соотношение этих параметров в выборке.
        
        condition_square = (df['Square'] > (df['LifeSquare'] + df['KitchenSquare']))
        
        df['TimesChanged'] = np.where(((df['LifeSquare'] > 250) | (df['LifeSquare'] < 10) | (df['LifeSquare'].isnull()) | condition_square), (df['TimesChanged'] + 1), df['TimesChanged'])
        df['LifeSquare'] = np.where((df['LifeSquare'] < 10), self.lifesquare_median, df['LifeSquare'])
        df['LifeSquare'] = np.where((df['LifeSquare'] > 250), self.lifesquare_mean, df['LifeSquare'])
        df['LifeSquare'] = np.where((condition_square | df['LifeSquare'].isnull()), (df['Square'] * self.life_to_square), df['LifeSquare'])
        
        # 6. Параметр KitchenSquare. Значения меньше 2 заменим на 2, а больше 50 на медиану.
        df['TimesChanged'] = np.where(((df['KitchenSquare'] < 2) | (df['KitchenSquare'] > 50)), (df['TimesChanged'] + 1), df['TimesChanged'])
        df['KitchenSquare'] = np.where((df['KitchenSquare'] < 2), 2, df['KitchenSquare'])
        df['KitchenSquare'] = np.where((df['KitchenSquare'] > 50), self.kitchensquare_median, df['KitchenSquare'])
        
        # 7. Параметр HouseFloor.
        df['TimesChanged'] = np.where(((df['HouseFloor'] > 60) | (df['HouseFloor'] < df['Floor'])), (df['TimesChanged'] + 1), df['TimesChanged'])
        df['HouseFloor'] = np.where((df['HouseFloor'] > 60), 60, df['HouseFloor'])
        
        # Теперь обработаем данные, где этаж больше этажности.
        # Для значений меньше 5 этажность заменим на 5, где меньше 10 на 10, где меньше 15 на 15, где меньше 20 на 20, 
        # а все, что больше на значение этажа.
        df['HouseFloor'] = np.where(((df['HouseFloor'] < df['Floor']) & (df['Floor'] <= 5)), 5, df['HouseFloor'])
        df['HouseFloor'] = np.where(((df['HouseFloor'] < df['Floor']) & (df['Floor'] <= 10)), 10, df['HouseFloor'])
        df['HouseFloor'] = np.where(((df['HouseFloor'] < df['Floor']) & (df['Floor'] <= 15)), 15, df['HouseFloor'])
        df['HouseFloor'] = np.where(((df['HouseFloor'] < df['Floor']) & (df['Floor'] <= 20)), 20, df['HouseFloor'])
        df['HouseFloor'] = np.where((df['HouseFloor'] < df['Floor']), df['Floor'], df['HouseFloor'])
        
        # 8. Параметр HouseYear. Всё, что больше 2020 года меняем на медианные значения.
        df['TimesChanged'] = np.where((df['HouseYear'] > 2020), (df['TimesChanged'] + 1), df['TimesChanged'])
        df['HouseYear'] = np.where((df['HouseYear'] > 2020), self.houseyear_median, df['HouseYear'])
        
        # 9. Параметр Healthcare_1. Удаляем из-за большого количества пропусков.
        df.drop(['Healthcare_1'], axis=1)
        
        # 10. Если в датасете до сих пор есть незаполненные значения, заполним их медианными значениями на всякий случай.
        df.fillna(self.medians, inplace=True)
        
        return df

In [ ]:
pipe = DataPipeline()
pipe.fit(df)
df = pipe.transform(df)

In [ ]:
df.hist(figsize=(20,20), bins=20)

In [ ]:
df.info()

### 3. Генерация новых признаков.

In [ ]:
feature_names = ['Id',
 'DistrictId',
 'Rooms',
 'Square',
 'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 'Ecology_2',
 'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 'Helthcare_2',
 'Shops_1',
 'Shops_2',
 'TimesChanged']

In [ ]:
plt.figure(figsize = (15,10))

sns.set(font_scale=1.4)

corr_matrix = df[feature_names].corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')

plt.title('Correlation matrix')
plt.show()

In [ ]:
class NewFeatures:
    
    """Генерация новых признаков, изменение старых"""
    
    def generate(self, df):
        
        """Изменение старых признаков"""
        
        # Поменяем год постройки на декаду по счету, начиная от 1910.
        df['HouseYear'] = np.where((df['HouseYear'] <= 1910), 1, df['HouseYear'])
        df['HouseYear'] = np.where(((1910 < df['HouseYear']) & (df['HouseYear'] <= 1920)), 2, df['HouseYear'])
        df['HouseYear'] = np.where(((1920 < df['HouseYear']) & (df['HouseYear'] <= 1930)), 3, df['HouseYear'])
        df['HouseYear'] = np.where(((1930 < df['HouseYear']) & (df['HouseYear'] <= 1940)), 4, df['HouseYear'])
        df['HouseYear'] = np.where(((1940 < df['HouseYear']) & (df['HouseYear'] <= 1950)), 5, df['HouseYear'])
        df['HouseYear'] = np.where(((1950 < df['HouseYear']) & (df['HouseYear'] <= 1960)), 6, df['HouseYear'])
        df['HouseYear'] = np.where(((1960 < df['HouseYear']) & (df['HouseYear'] <= 1970)), 7, df['HouseYear'])
        df['HouseYear'] = np.where(((1970 < df['HouseYear']) & (df['HouseYear'] <= 1980)), 8, df['HouseYear'])
        df['HouseYear'] = np.where(((1980 < df['HouseYear']) & (df['HouseYear'] <= 1990)), 9, df['HouseYear'])
        df['HouseYear'] = np.where(((1990 < df['HouseYear']) & (df['HouseYear'] <= 2000)), 10, df['HouseYear'])
        df['HouseYear'] = np.where(((2000 < df['HouseYear']) & (df['HouseYear'] <= 2010)), 11, df['HouseYear'])
        df['HouseYear'] = np.where(((2010 < df['HouseYear']) & (df['HouseYear'] <= 2020)), 12, df['HouseYear'])
        
        """Генерация новых признаков"""
        
        # Соотношение этажа к этажности. Если квартира находится ближе к первому этажу, то значение будет ближе к 0, если к последнему, то к единице.
        df['FirstLastFloor'] = df['Floor'] / df['HouseFloor']
        
        # Средняя площадь комнат.
        df['RoomSquare'] = df['LifeSquare'] / df['Rooms']
        
        # Из-за сильной корреляции признаков сделаем новый на основе их отношения друг к другу.
        df['Social'] = df['Social_1'] / df['Social_2']
        
        return df

In [ ]:
feat = NewFeatures()
df = feat.generate(df)

In [ ]:
target_name = 'Price'
all_features = ['Id',
 'DistrictId',
 'Rooms',
 'Square',
 'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 'Ecology_2',
 'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 'Helthcare_2',
 'Shops_1',
 'Shops_2',
 'TimesChanged',
 'FirstLastFloor',
 'RoomSquare',
 'Social']

In [ ]:
plt.figure(figsize = (15,10))

sns.set(font_scale=1.4)

corr_matrix = df[feature_names].corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')

plt.title('Correlation matrix')
plt.show()

### 4. Оценка модели перед корректировками.

In [ ]:
X = df[all_features]
y = df[target_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
gb_model = GradientBoostingRegressor(criterion='mse',
                                     max_depth=5,
                                     min_samples_leaf=9,
                                     random_state=42,  
                                     n_estimators=200)
gb_model.fit(X_train, y_train)

y_train_preds = gb_model.predict(X_train)

y_test_preds = gb_model.predict(X_test)

In [ ]:
evaluate_preds(y_train, y_train_preds, y_test, y_test_preds)

### 5. Корректировки и финальные результаты.

In [ ]:
cv_score = cross_val_score(gb_model, X, y, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=21))
cv_score

In [ ]:
feature_importances = pd.DataFrame(zip(X_train.columns, 
                                       gb_model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False, inplace=True)
feature_importances

In [ ]:
final_features = ['Id',
 'DistrictId',
 'Rooms',
 'Square',
 'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 'Social_1',
 'Social_2',
 'Social_3',
 'Helthcare_2',
 'Shops_1',
 'TimesChanged',
 'FirstLastFloor',
 'RoomSquare',
 'Social']

In [ ]:
X = df[final_features]
y = df[target_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
gb_model = GradientBoostingRegressor(criterion='mse',
                                     max_depth=5,
                                     min_samples_leaf=9,
                                     random_state=42,  
                                     n_estimators=200)
gb_model.fit(X_train, y_train)

y_train_preds = gb_model.predict(X_train)

y_test_preds = gb_model.predict(X_test)

In [ ]:
evaluate_preds(y_train, y_train_preds, y_test, y_test_preds)

### 6. Прогнозирование на тестовом датасете.

In [ ]:
test_directory = '/home/maxim/Документы/GeekBrains/1 четверть. Библиотки Python для Data Science. Numpy, Matplotlib, Scikit-learn/Manuals/test.csv'

In [ ]:
new_file = '/home/maxim/Документы/GeekBrains/1 четверть. Библиотки Python для Data Science. Numpy, Matplotlib, Scikit-learn/Manuals/prices_test.csv'

In [ ]:
df_test = pd.read_csv(test_directory)
df_test = pipe.transform(df_test)

In [ ]:
df_test = feat.generate(df_test)

In [ ]:
predictions = gb_model.predict(df_test[final_features])
predictions

In [ ]:
df_test.head()

In [ ]:
y_predict_id = pd.DataFrame(df_test, columns=['Id'])
y_predict_2 = pd.DataFrame(predictions, columns=['Price'])
new_prices = pd.concat([y_predict_id, y_predict_2], axis=1)
new_prices.to_csv('/home/maxim/Документы/GeekBrains/1 четверть. Библиотки Python для Data Science. Numpy, Matplotlib, Scikit-learn/Tasks/v.1.5.csv', index=False)